In [127]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# Set the path to the folder containing audio files
audio_folder = "4way-tabla-ismir21-dataset/train/audios"  # Replace with the path to your audio files folder

# Set the parameters for Mel spectrogram
n_fft = 512 # Number of points in each FFT
hop_length = n_fft  # Hop size
fft_len = n_fft//2 + 1  # Number of Mel bands
window = "hann"  # Window function
stft_taken = fft_len//4

# Initialize an empty matrix
stft_matrix = []
flux_arr = []
mfcc_matrix = []
centroids = []
centroid_matrix = []
segments_matrix = []
max_freq = []

# Iterate through audio files in the folder
for filename in sorted(os.listdir(audio_folder)):
    if filename.endswith(".wav"):  # Adjust the file extension if needed
        audio_path = os.path.join(audio_folder, filename)

        # Load the audio file
        y, sr = librosa.load(audio_path)
        if sr != 22050:
            print(sr)
        
        # num_samples = len(y)
        # num_segments = num_samples // n_fft
        # segments = [y[i * n_fft:(i + 1) * n_fft] for i in range(num_segments)]
        # if(num_samples % n_fft != 0):
        #     segment = y[num_segments * n_fft:]
        #     segment = segment.tolist()
        #     for i in range(n_fft - num_samples % n_fft):
        #         segment.append(0)
        #     segments.append(segment)

        # segments_matrix.extend(segments)

        mfcc = librosa.feature.mfcc(y=y, n_mfcc=13, sr=sr, n_fft=n_fft, hop_length=hop_length)
        stft = np.abs(librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, window=window))
        flux = librosa.onset.onset_strength(y=y, sr=sr, n_fft=n_fft, hop_length = hop_length)
        flux = flux
        
        stft_matrix.append(stft[:stft_taken,:])
        flux_arr.append(flux)
        mfcc_matrix.append(mfcc)
        


#segments_matrix = np.concatenate(segments_matrix)
stft_matrix = np.hstack(stft_matrix)
flux_arr = np.hstack(flux_arr)
mfcc_matrix = np.hstack(mfcc_matrix)


for i in range(stft_matrix.shape[1]):
    segment = stft_matrix[:, i]
    
    # Compute the frequencies and magnitudes
    frequencies = librosa.fft_frequencies(sr=sr, n_fft=n_fft)[:stft_taken]
    magnitudes = segment
    # # Transpose frequencies to match the shape of magnitudes
    # frequencies = frequencies[:, np.newaxis]
    max_frequency_index = np.argmax(segment, axis=0)
    max_freq.append(frequencies[max_frequency_index])
    # Calculate the spectral centroid

    centroid = np.sum(frequencies * magnitudes, axis=0) / np.sum(magnitudes, axis=0)
    # Take the mean of the centroid values across the segment
    mean_centroid = np.mean(centroid)
    centroid_matrix.append(mean_centroid)

flux_arr = flux_arr.T
stft_matrix = stft_matrix.T

mfcc_matrix = mfcc_matrix.T
max_freq = np.array(max_freq)
centroid_matrix = np.array(centroid_matrix)


NameError: name 'os' is not defined

In [217]:
# print(segment.shape)
print(frequencies.shape)
print(magnitudes.shape)
print(max_freq.shape)

(64,)
(64,)
(202194,)


In [218]:
print(mfcc_matrix.shape)
print(flux_arr.shape)
print(centroid_matrix.shape)
# print(segments_matrix.shape)

(202194, 13)
(202194,)
(202194,)


In [129]:
# segments_matrix = np.array(segments_matrix)

In [131]:
# import time

# # Record the start time
# start_time = time.time()
# autocorrelation_matrix[0, 0] = np.correlate(segments_matrix[0], segments_matrix[0], mode='full')[0]
# # Record the end time
# end_time = time.time()

# # Calculate the elapsed time
# elapsed_time = end_time - start_time

# print(f"Time taken: {elapsed_time} seconds")

In [132]:
# num_segments = segments_matrix.shape[0]
# autocorrelation_matrix = np.zeros((num_segments, num_segments))

# for i in range(num_segments):
#     for j in range(i,num_segments):
#         autocorrelation_matrix[i, j] = np.correlate(segments_matrix[i], segments_matrix[j], mode='full')[0]
#         autocorrelation_matrix[j,i] = autocorrelation_matrix[i,j]

In [ ]:
# num_segments = segments_matrix.shape[0]
# autocorrelation_matrix = np.zeros((num_segments, num_segments))
# for i in range(num_segments):
#     for j in range(i,num_segments):
#         result = np.fft.ifft(np.fft.fft(segments_matrix[i]) * np.fft.fft(segments_matrix[j]).conj()).real
#         autocorrelation_matrix[i, j] = np.sum(result)

In [219]:

features = np.column_stack((mfcc_matrix, flux_arr*4, ))

In [190]:
print(flux_arr)
print(flux_arr.max())
print(flux_arr.min())

[0.        0.        0.        ... 0.607357  1.1710113 0.9910203]
44.454247
0.0


In [191]:
print(centroid_matrix)
print(centroid_matrix.max())
print(centroid_matrix.min())

[1365.09812052 1469.53397279 1381.88916934 ...  171.80083917  163.47974022
  168.5204392 ]
1848.0559591116091
37.71570449081577


In [220]:
num_clusters = 4

In [221]:
# Define the time_step of the audio file
time_step = hop_length/22050

audio_folder = "4way-tabla-ismir21-dataset/train/audios" 
onset_folder = "4way-tabla-ismir21-dataset/train/onsets"

# Initialize an empty matrix to store the spectrograms
label_array = []
lens = []

for filename in sorted(os.listdir(audio_folder)):

    file_name, extension = os.path.splitext(filename)
    onsetfile = f"{file_name}.{'onsets'}"

    # Provide the path to your audio file
    audio_file_path = os.path.join(audio_folder, filename)
    
    # Load the audio file
    y, sr = librosa.load(audio_file_path)

    # Calculate the duration of the audio in seconds
    audio_duration = librosa.get_duration(y=y, sr=sr)


    # Calculate the number of time steps and initialize the array with zeros
    num_time_steps = int(audio_duration / time_step)
    num_time_steps += 1
    stroke_array = np.zeros(num_time_steps)
    
    # Read the .ONSETS file to get the times of the strokes
    for index, i in enumerate(["b", "d", "rb", "rt"]):
        file_path = os.path.join(onset_folder, i, onsetfile)
        with open(file_path, 'r') as file:
            stroke_times = [float(line.strip()) for line in file]

        # Set the values to 1 where there is a stroke within the time period
        for stroke_time in stroke_times:
            time_step_index = int(stroke_time / time_step)
            if time_step_index < num_time_steps:
                stroke_array[time_step_index] = index + 1
        
    
    label_array.append(stroke_array)
    lens.append(len(stroke_array))

label_array = np.hstack(label_array)

label_array = label_array.T   


In [222]:
print(features.shape)
print(label_array.shape)

(202194, 14)
(202194,)


In [223]:
pos_labels = label_array[label_array != 0]
pos_features = features[label_array != 0][:]

print(pos_features.shape)
print(pos_labels.shape)

(25654, 14)
(25654,)


In [224]:
pos_labels = pos_labels - 1
pos_labels.astype(int)

array([1, 1, 3, ..., 1, 1, 0])

In [225]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(pos_features)

In [226]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
y_pred_kmeans = kmeans.fit_predict(features_scaled)
ari = adjusted_rand_score(pos_labels, y_pred_kmeans)
print(f"Adjusted Rand Index (ARI): {ari}")

c:\Users\chinm\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Adjusted Rand Index (ARI): 0.007933165406509093


In [28]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
y_pred_kmeans = kmeans.fit_predict(features_scaled)
ari = adjusted_rand_score(pos_labels, y_pred_kmeans)
print(f"Adjusted Rand Index (ARI): {ari}")

0.04873491632785393

In [46]:
dbscan = DBSCAN(eps=1.525, min_samples=2) 
y_pred_dbscan = dbscan.fit_predict(features_scaled)
ari = adjusted_rand_score(pos_labels, y_pred_dbscan)
print(f"Adjusted Rand Index (ARI): {ari}")

Adjusted Rand Index (ARI): 0.022156200140686807


In [47]:
from sklearn.cluster import AgglomerativeClustering

In [48]:
agglomerative = AgglomerativeClustering(n_clusters=num_strokes)
y_pred_agglomerative = agglomerative.fit_predict(features_scaled)

# Evaluate the clustering results using Adjusted Rand Index (ARI)
ari_agglomerative = adjusted_rand_score(pos_labels, y_pred_agglomerative)
print(f"Adjusted Rand Index (ARI) for Agglomerative Clustering: {ari_agglomerative}")

Adjusted Rand Index (ARI) for Agglomerative Clustering: 0.03815880441380059
